不動産価格予測

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install h2o >> /dev/null

In [3]:
!pip install optuna xfeat japanize-matplotlib >> /dev/null

In [4]:
# パスの設定
My_PATH = "/content/drive/MyDrive/property_values_prediction"

%cd {My_PATH}

/content/drive/MyDrive/property_values_prediction


In [5]:
# zipファイルの解凍

# 古いdata.zipの削除
!rm -r /content/drive/MyDrive/property_values_prediction/data

import zipfile
with zipfile.ZipFile('data.zip', 'r')as f:
    f.extractall('./data')

In [6]:
# train.zipの解凍
# 古いtrain.zipの削除
!rm -r /content/drive/MyDrive/property_values_prediction/data/train

with zipfile.ZipFile('data/train.zip', 'r')as f:
    f.extractall('./data/')

!rm -r /content/drive/MyDrive/property_values_prediction/data/__MACOSX

rm: cannot remove '/content/drive/MyDrive/property_values_prediction/data/train': No such file or directory


In [7]:
import datetime
import os

now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d-%H-%M")
dir_for_output = "./output/" + current_time

os.makedirs(dir_for_output, exist_ok=True)

In [8]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
import pandas as pd
import seaborn as sns
sns.set(font="IPAexGothic")

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import (
    SelectCategorical,
    LabelEncoder,
    LambdaEncoder,
    Pipeline,
    ConcatCombination,
    SelectNumerical,
    ArithmeticCombinations,
    TargetEncoder,
    aggregation,
    GBDTFeatureSelector,
    GBDTFeatureExplorer,
)

h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmptbkhmxct
  JVM stdout: /tmp/tmptbkhmxct/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmptbkhmxct/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 22 days
H2O_cluster_name:,H2O_from_python_unknownUser_dfzghf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [9]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(moyori)
    return moyori

def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

コンペデータ作成（利用カラム、目的変数など決める）

dataディレクトリ配下に以下のようにデータを格納しているとする

├─ input<br>
│   ├─ data_explanation.xlsx<br>
│   ├─ sample_submission.csv<br>
│   ├─ test.csv<br>
│   └─ train<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 01.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 02.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 03.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ ...<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└─ 47.csv<br>
└─ output


In [10]:
BASE_PATH = './data/'
RANDOM_STATE = 10

In [11]:
paths = glob(BASE_PATH + 'train/*')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)
test_df = pd.read_csv(BASE_PATH + 'test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.



In [12]:
train_df.isnull().sum()

ID                   0
種類                   0
地域              677392
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                660
最寄駅：名称            2672
最寄駅：距離（分）        23098
間取り              23661
面積（㎡）                0
土地の形状           677392
間口              677392
延床面積（㎡）         677392
建築年              19622
建物の構造            16486
用途               58246
今後の利用目的         364049
前面道路：方位         677392
前面道路：種類         677392
前面道路：幅員（ｍ）      677392
都市計画             19221
建ぺい率（％）          23345
容積率（％）           23345
取引時点                 0
改装               61966
取引の事情等          658854
取引価格（総額）_log         0
dtype: int64

In [13]:
sub_df = pd.read_csv(BASE_PATH + 'sample_submission.csv')

In [14]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
rm_cols = []

In [15]:
df = pd.concat([train_df, test_df])

rm_cols += ['市区町村コード']
for i,v in df.nunique().iteritems():
    if v <= 1:
        rm_cols.append(i)

rm_cols

['市区町村コード',
 '種類',
 '地域',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）']

In [16]:
test_df[TARGET] = np.nan
train_df.drop(rm_cols, axis=1, inplace=True)
test_df.drop(rm_cols, axis=1, inplace=True)
df = pd.concat([train_df, test_df])
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(700543, 19)

In [17]:
df['取引時点'].unique()

array(['2005年第３四半期', '2005年第４四半期', '2006年第１四半期', '2006年第２四半期',
       '2006年第３四半期', '2006年第４四半期', '2007年第１四半期', '2007年第２四半期',
       '2007年第３四半期', '2007年第４四半期', '2008年第１四半期', '2008年第２四半期',
       '2008年第３四半期', '2008年第４四半期', '2009年第１四半期', '2009年第２四半期',
       '2009年第３四半期', '2009年第４四半期', '2010年第１四半期', '2010年第２四半期',
       '2010年第３四半期', '2010年第４四半期', '2011年第１四半期', '2011年第２四半期',
       '2011年第３四半期', '2011年第４四半期', '2012年第１四半期', '2012年第２四半期',
       '2012年第３四半期', '2012年第４四半期', '2013年第１四半期', '2013年第２四半期',
       '2013年第３四半期', '2013年第４四半期', '2014年第１四半期', '2014年第２四半期',
       '2014年第３四半期', '2014年第４四半期', '2015年第１四半期', '2015年第２四半期',
       '2015年第３四半期', '2015年第４四半期', '2016年第１四半期', '2016年第２四半期',
       '2016年第３四半期', '2016年第４四半期', '2017年第１四半期', '2017年第２四半期',
       '2017年第３四半期', '2017年第４四半期', '2018年第１四半期', '2018年第２四半期',
       '2018年第３四半期', '2018年第４四半期', '2019年第１四半期', '2019年第２四半期',
       '2019年第３四半期', '2019年第４四半期', '2020年第１四半期', '2020年第２四半期',
       '2020年第３四半期', '2020年第４四半期', '2021年第１四半期'], dtype

In [18]:
val_min_idx = min(df[df['取引時点'].str.contains('2020年第２四半期|2020年第３四半期', regex=True)].index)
test_min_idx = min(df[df['取引時点'].str.contains('2020年第４四半期|2021年第１四半期', regex=True)].index)
val_min_idx, test_min_idx

(652493, 677392)

In [19]:
set(df.iloc[val_min_idx:test_min_idx, :]['取引時点'].values)

{'2020年第２四半期', '2020年第３四半期'}

特徴量生成

In [20]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

In [21]:
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836443,
 '京都府': 7.15593584855535,
 '佐賀県': 7.009937258263499,
 '兵庫県': 7.157742870483325,
 '北海道': 6.965967793677889,
 '千葉県': 7.155825324965048,
 '和歌山県': 7.0226902108157585,
 '埼玉県': 7.169744271396765,
 '大分県': 6.911537826573121,
 '大阪府': 7.158352973821433,
 '奈良県': 7.0568846102346985,
 '宮城県': 7.03954425380504,
 '宮崎県': 6.958678653711221,
 '富山県': 7.046920071386999,
 '山口県': 7.045342730349815,
 '山形県': 7.087990602814885,
 '山梨県': 6.848839888686703,
 '岐阜県': 7.078095913486438,
 '岡山県': 7.038090734927505,
 '岩手県': 6.92500267906058,
 '島根県': 7.16537621470031,
 '広島県': 7.09344902165506,
 '徳島県': 6.912051981629739,
 '愛媛県': 6.9664330115966635,
 '愛知県': 7.107301969238665,
 '新潟県': 6.913835156235099,
 '東京都': 7.377929204637786,
 '栃木県': 6.950636958071022,
 '沖縄県': 7.160046604428345,
 '滋賀県': 7.166026958168428,
 '熊本県': 6.970182251843288,
 '石川県': 6.929363348877184,
 '神奈川県': 7.258118267209654,
 '福井県': 6.979575815840192,
 '福岡県': 6.9922649050422665,
 '福島県': 6.974380579140183,
 '秋田県': 6.94131374711262,
 

In [22]:
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)
df.head(2)

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te
0,27232131,大阪府,大阪市旭区,新森,森小路,6,１ＬＤＫ,50,昭和49年,ＲＣ,住宅,NaN,準工業地域,80.0,200.0,2005年第３四半期,改装済,NaN,7.041393,0,0.0
1,14289301,神奈川県,川崎市高津区,千年新町,武蔵新城,7,３ＬＤＫ,60,平成4年,ＲＣ,住宅,NaN,第１種中高層住居専用地域,60.0,200.0,2005年第３四半期,改装済,NaN,7.361728,0,0.0


In [23]:
df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2021-int(x[:4]))
df.drop(['取引時点'], axis=1, inplace=True)
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

In [24]:
num_df = SelectNumerical().fit_transform(df)
num_df.head(2)

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前
0,27232131,6.0,50,1974.0,80.0,200.0,7.041393,0,0.0,16
1,14289301,7.0,60,1992.0,60.0,200.0,7.361728,0,0.0,16


In [25]:
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head(2)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等
0,0,0,0,0,0,0,0,-1,0,0,-1
1,1,1,1,1,1,0,0,-1,1,0,-1


In [26]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"], 
            drop_origin=True, 
            operator="*", 
            r=2,
        ),
    ]
)

num_comb_df = encoder.fit_transform(df)/100
num_comb_df.head(2)

,面積（㎡）容積率（％）_combi
0,100.0
1,120.0


In [27]:
agg_dfs = []

def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                        group_key=group_col,
                        group_values=['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
                        agg_methods=['count', 'mean', 'min', 'max'],
                        )

    return agg_df[agg_cols]

group_col = '市区町村名'
agg_dfs.append(get_agg_df(df, group_col))
agg_dfs[0].head(2)

,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,789,808,806,806,8.206591,59.814356,70.397022,236.228288,0.0,15,60.0,200.0,23.0,115,80.0,400.0
1,2945,3089,2872,2872,13.020713,59.043380,61.922006,212.580084,0.0,15,40.0,80.0,105.0,550,80.0,1100.0


In [28]:
feat_df = pd.concat([num_df,le_df,num_comb_df]+agg_dfs, axis=1)
print(feat_df.shape)

(700543, 38)


In [29]:
feat_df.dtypes

ID                                   int64
最寄駅：距離（分）                          float64
面積（㎡）                                int64
建築年                                float64
建ぺい率（％）                            float64
容積率（％）                             float64
取引価格（総額）_log                       float64
取引時点_enc                             int64
都道府県名_te                           float64
取引時点_何年前                             int64
都道府県名                                int64
市区町村名                                int64
地区名                                  int64
最寄駅：名称                               int64
間取り                                  int64
建物の構造                                int64
用途                                   int64
今後の利用目的                              int64
都市計画                                 int64
改装                                   int64
取引の事情等                               int64
面積（㎡）容積率（％）_combi                  float64
agg_count_最寄駅：距離（分）_grpby_市区町村名      int64
agg_count_面

モデル構築

In [30]:
train_df = feat_df.iloc[:val_min_idx, :]
val_df = feat_df.iloc[val_min_idx:test_min_idx, :]
test_df = feat_df.iloc[test_min_idx:, :]
print(train_df.shape, val_df.shape, test_df.shape)

(652493, 38) (24899, 38) (23151, 38)


In [31]:
train_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,27232131,6.0,50,1974.0,80.0,200.0,7.041393,0,0.0,16,0,0,0,0,0,0,0,-1,0,0,-1,100.0,789,808,806,806,8.206591,59.814356,70.397022,236.228288,0.0,15,60.0,200.0,23.0,115,80.0,400.0
1,14289301,7.0,60,1992.0,60.0,200.0,7.361728,0,0.0,16,1,1,1,1,1,0,0,-1,1,0,-1,120.0,2945,3089,2872,2872,13.020713,59.043380,61.922006,212.580084,0.0,15,40.0,80.0,105.0,550,80.0,1100.0
2,13417040,NaN,60,1993.0,60.0,200.0,7.255273,0,0.0,16,2,2,2,2,2,0,0,-1,0,0,-1,120.0,4605,4893,4812,4812,9.963084,50.796035,64.170823,279.640482,0.0,15,40.0,80.0,45.0,270,80.0,600.0
3,14267697,NaN,85,1982.0,NaN,NaN,7.556303,0,0.0,16,1,3,3,3,1,-1,0,-1,-1,-1,-1,NaN,3465,3675,3375,3375,13.894372,67.380952,59.454815,170.755556,0.0,15,30.0,60.0,120.0,210,80.0,500.0
4,13386603,5.0,20,2005.0,80.0,500.0,7.278754,0,0.0,16,2,4,4,4,3,0,0,-1,2,1,-1,100.0,8377,8625,8526,8526,8.184075,46.059710,66.272578,274.364297,0.0,10,30.0,80.0,75.0,220,80.0,600.0


In [32]:
test_df = test_df.fillna(0)
test_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
677392,14068534,11.0,60,1996.0,60.0,200.0,0.0,61,7.268224,1,1,251,5181,1999,2,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677393,14068533,24.0,70,1998.0,50.0,100.0,0.0,61,7.268224,1,1,251,5181,1999,1,0,0,0,9,0,-1,70.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677394,14068532,11.0,60,1996.0,60.0,200.0,0.0,61,7.268224,1,1,251,5181,1999,1,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677395,14068420,21.0,105,1969.0,60.0,200.0,0.0,61,7.268224,1,1,251,5018,1702,19,0,0,0,1,0,-1,210.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0
677396,14068219,21.0,65,1979.0,60.0,200.0,0.0,61,7.268224,1,1,251,4480,1544,1,0,-1,0,7,1,-1,130.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0


In [33]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

In [34]:
cat_cols = list(le_df.columns) + ['取引時点_enc']
cat_cols

['都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '間取り',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '都市計画',
 '改装',
 '取引の事情等',
 '取引時点_enc']

In [35]:
train_x = train_df[feat_cols]
train_y = train_df[TARGET]
val_x = val_df[feat_cols]
val_y = val_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

In [36]:
train = pd.merge(train_x, train_y, left_index=True, right_index=True)
valid = pd.merge(val_x, val_y, left_index=True, right_index=True)
test = pd.merge(test_x, test_y, left_index=True, right_index=True)

In [37]:
test.head()

,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名,取引価格（総額）_log
677392,11.0,60,1996.0,60.0,200.0,61,7.268224,1,1,251,5181,1999,2,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677393,24.0,70,1998.0,50.0,100.0,61,7.268224,1,1,251,5181,1999,1,0,0,0,9,0,-1,70.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677394,11.0,60,1996.0,60.0,200.0,61,7.268224,1,1,251,5181,1999,1,1,-1,0,0,1,-1,120.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677395,21.0,105,1969.0,60.0,200.0,61,7.268224,1,1,251,5018,1702,19,0,0,0,1,0,-1,210.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0
677396,21.0,65,1979.0,60.0,200.0,61,7.268224,1,1,251,4480,1544,1,0,-1,0,7,1,-1,130.0,2732,2736,2711,2711,12.015739,58.961988,64.452232,239.933604,0.0,15,40.0,80.0,75.0,165,80.0,900.0,0.0


In [38]:
h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)
h2o_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [39]:
# 説明変数/目的変数のカラムを指定
kw = h2o.H2OFrame(list(h2o_train.columns))

feature_cols = kw.columns[:-1]
target_cols = "取引価格（総額）_log"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [43]:
dl_fit3 = H2ODeepLearningEstimator(
    model_id='dl_fit3', 
    epochs=200, 
    hidden=[50,50],
    score_interval=1,          #used for early stopping
    stopping_rounds=30,         #used for early stopping
    stopping_metric='MAE',     #used for early stopping
    # stopping_tolerance=0.0005, #used for early stopping
    seed=1
    )
dl_fit3.train(x=feat_cols, y=TARGET, training_frame=h2o_train, validation_frame=h2o_valid)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  dl_fit3


Status of Neuron Layers: predicting 取引価格（総額）_log, regression, gaussian distribution, Quadratic loss, 4,451 weights/biases, 67.5 KB, 29,714,358 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,36,Input,0,,,,,,,,,
1,,2,50,Rectifier,0,0,0,0.00804517,0.0108606,0,-0.0220692,1.03518,0.0222794,1.63411
2,,3,50,Rectifier,0,0,0,0.00196464,0.0017432,0,-0.0582334,0.414963,1.00284,0.631884
3,,4,1,Linear,,0,0,0.00012058,6.69334e-05,0,0.0121366,0.0984131,-1.38307,1.09713e-154




ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.01967938103081309
RMSE: 0.1402832172100893
MAE: 0.09119995796637134
RMSLE: 0.017814323203385753
Mean Residual Deviance: 0.01967938103081309

ModelMetricsRegression: deeplearning
** Reported on validation data. **

MSE: 0.021371878355710434
RMSE: 0.14619123898411435
MAE: 0.09360408716769048
RMSLE: 0.01881898166082274
Mean Residual Deviance: 0.021371878355710434

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2021-11-29 13:46:14,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2021-11-29 13:46:17,4.607 sec,16689 obs/sec,0.085072,1,55509.0,0.189237,0.035811,0.140693,0.709219,0.217067,0.047118,0.169595,0.624929
2,,2021-11-29 13:46:21,8.683 sec,15491 obs/sec,0.169442,2,110560.0,0.174659,0.030506,0.125041,0.752295,0.184217,0.033936,0.134161,0.729864
3,,2021-11-29 13:46:24,11.829 sec,16538 obs/sec,0.254433,3,166016.0,0.164958,0.027211,0.115923,0.779047,0.183158,0.033547,0.133942,0.732960
4,,2021-11-29 13:46:27,14.676 sec,17508 obs/sec,0.339034,4,221217.0,0.163746,0.026813,0.115946,0.782283,0.181204,0.032835,0.133258,0.738627
5,,2021-11-29 13:46:30,17.465 sec,18285 obs/sec,0.423603,5,276398.0,0.158994,0.025279,0.109893,0.794736,0.176415,0.031122,0.128471,0.752260
6,,2021-11-29 13:46:34,21.377 sec,17654 obs/sec,0.507872,6,331383.0,0.157865,0.024921,0.108678,0.797640,0.174355,0.030400,0.127197,0.758012
7,,2021-11-29 13:46:37,24.623 sec,17989 obs/sec,0.592581,7,386655.0,0.159617,0.025478,0.112358,0.793124,0.177366,0.031459,0.131384,0.749582
8,,2021-11-29 13:46:43,30.534 sec,18296 obs/sec,0.761994,9,497196.0,0.153722,0.023630,0.104679,0.808123,0.166065,0.027578,0.117869,0.780477
9,,2021-11-29 13:46:46,33.190 sec,18733 obs/sec,0.846748,10,552497.0,0.151904,0.023075,0.103878,0.812634,0.164760,0.027146,0.116418,0.783914



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,都道府県名_te,1.000000,1.000000,0.143131
1,都道府県名,0.382503,0.382503,0.054748
2,agg_min_最寄駅：距離（分）_grpby_市区町村名,0.379614,0.379614,0.054335
3,agg_mean_建ぺい率（％）_grpby_市区町村名,0.320067,0.320067,0.045812
4,agg_mean_面積（㎡）_grpby_市区町村名,0.319594,0.319594,0.045744
5,市区町村名,0.289212,0.289212,0.041395
6,agg_max_容積率（％）_grpby_市区町村名,0.273170,0.273170,0.039099
7,面積（㎡）,0.260621,0.260621,0.037303
8,agg_mean_最寄駅：距離（分）_grpby_市区町村名,0.257898,0.257898,0.036913
9,agg_mean_容積率（％）_grpby_市区町村名,0.256340,0.256340,0.036690



See the whole table with table.as_data_frame()


In [44]:
dl_perf3 = dl_fit3.model_performance(h2o_valid)

In [45]:
print(dl_perf3.mae())

0.09360408716769048


In [62]:
h2o.mojo_predict_pandas

AttributeError: ignored

In [60]:
# 予測
y_pred = dl_perf3.predict(h2o_test).as_data_frame()
y_pred.index = test_df.index
y_pred

AttributeError: ignored